In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '487-final-project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

['eval.py', 'README.md', 'test.py', 'naivebayes.py', 'scrape.py', 'preprocess.py', 'requirements.txt', 'data', '__pycache__', 'checkpoint.pth', 'checkpoint_7.pth', 'model.py', 'train_model.ipynb']


In [3]:
!pip install transformers
from transformers import BertTokenizer
import torch
import numpy as np
import pandas as pd
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
from google.colab import files

     |████████████████████████████████| 3.8 MB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 50.9 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 895 kB 58.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from model import train
from model import BertClassifier

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Dataset

In [6]:
np.random.seed(112)

df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, 'data/MBIC/labeled_dataset.csv'))

df[['type', 'article']]
df = df.dropna()

df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.8*len(df)), int(.9*len(df))])
print(df_train.shape[0], df_val.shape[0], df_test.shape[0])

1260 158 158


# Hyperparameter Tuning

# Train Model

In [7]:
EPOCHS = 10
clf = BertClassifier()
LR = 1e-6

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
train(clf, df_train, df_val, LR, EPOCHS)

torch.save(clf.state_dict(), os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_2.pth'))

files.download(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint_2.pth'))

Using Cuda: True


100%|██████████| 630/630 [01:26<00:00,  7.25it/s]


Epochs: 1 | Train Loss:  0.535         | Train Accuracy:  0.441         | Val Loss:  0.518         | Val Accuracy:  0.513


100%|██████████| 630/630 [01:26<00:00,  7.25it/s]


Epochs: 2 | Train Loss:  0.482         | Train Accuracy:  0.610         | Val Loss:  0.449         | Val Accuracy:  0.658


...

Epochs: 9 | Train Loss:  0.099         | Train Accuracy:  0.967         | Val Loss:  0.220         | Val Accuracy:  0.854


100%|██████████| 630/630 [01:25<00:00,  7.34it/s]


Epochs: 10 | Train Loss:  0.089         | Train Accuracy:  0.967         | Val Loss:  0.221         | Val Accuracy:  0.861


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Test Model

In [35]:
from model import Dataset
from sklearn.metrics import precision_recall_fscore_support

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model_test = BertClassifier()
state_dict = torch.load(os.path.join(GOOGLE_DRIVE_PATH, 'checkpoint.pth'))
model_test.load_state_dict(state_dict)
model_test.to(device)

test_data = Dataset(df_test)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=2, shuffle=True)

total_acc_test = 0
preds = []
labels = []
with torch.no_grad():
  for test_input, test_label in tqdm(test_dataloader):

      test_label = test_label.to(device)
      mask = test_input['attention_mask'].to(device)
      input_id = test_input['input_ids'].squeeze(1).to(device)

      output = model_test(input_id, mask)
      acc = (output.argmax(dim=1) == test_label).sum().item()
      preds.extend(output.argmax(dim=1).tolist())
      labels.extend(test_label.tolist())
      total_acc_test += acc
      
print()
print()
print("Test Accuracy of", round(total_acc_test/len(df_test), 3))
print()
preds = np.asarray(preds)
labels = np.asarray(labels)
mac_p, mac_r, mac_f1, _ = precision_recall_fscore_support(preds, labels, average='macro')
mic_p, mic_r, mic_f1, _ = precision_recall_fscore_support(preds, labels, average='micro')
print('Macro:')
print(f'Precision: {round(mac_p, 3)}, Recall: {round(mac_r, 3)}, F1-Score: {round(mac_f1, 3)}')
print("-----------------")
print('Micro:')
print(f'Precision: {round(mic_p, 3)}, Recall: {round(mic_r, 3)}, F1-Score: {round(mic_f1, 3)}')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 79/79 [00:03<00:00, 23.48it/s]



Test Accuracy of 0.956

Macro:
Precision: 0.948, Recall: 0.965, F1-Score: 0.956
-----------------
Micro:
Precision: 0.956, Recall: 0.956, F1-Score: 0.956
